In [1]:
import pandas as pd
import geopandas as gpd
import requests, json, geojson
from shapely.geometry import shape
import shapely.wkt
import os

import sys
sys.path.append('../scripts/')
from read_utils import read_file, create_folder, temp_record_query, temp_record_sdf

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


- postcode(shapefile)

In [2]:
poa_sf = gpd.read_file('../data/tables/external_POA/POA_2021_AUST_GDA2020.shp')
poa_sf['geometry'] = poa_sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
poa_gdf = gpd.GeoDataFrame(poa_sf)

In [3]:
poa_gdf.head()

,POA_CODE21,POA_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,SHAPE_Leng,SHAPE_Area,geometry
0,0800,0800,AUS,Australia,3.1731,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.081893,0.000264,"POLYGON ((130.83681 -12.45376, 130.83684 -12.4..."
1,0810,0810,AUS,Australia,24.4283,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.241859,0.002031,"POLYGON ((130.89986 -12.36567, 130.89875 -12.3..."
2,0812,0812,AUS,Australia,35.8899,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.278788,0.002983,"POLYGON ((130.91915 -12.40786, 130.91831 -12.4..."
3,0820,0820,AUS,Australia,39.0642,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.409134,0.003248,"POLYGON ((130.85260 -12.43994, 130.85089 -12.4..."
4,0822,0822,AUS,Australia,150775.8030,http://linked.data.gov.au/dataset/asgsed3/POA/...,90.601831,12.564238,"MULTIPOLYGON (((136.56648 -12.08392, 136.56704..."


In [4]:
poa_gdf = poa_gdf.replace(to_replace='None', value=0).dropna()

In [5]:
poa_gdf['longitude'] = poa_gdf['geometry'].apply(lambda x: x.centroid.x)

In [6]:
poa_gdf['latitude'] = poa_gdf['geometry'].apply(lambda x: x.centroid.y)

In [7]:
poa_gdf.head()

,POA_CODE21,POA_NAME21,AUS_CODE21,AUS_NAME21,AREASQKM21,LOCI_URI21,SHAPE_Leng,SHAPE_Area,geometry,longitude,latitude
0,0800,0800,AUS,Australia,3.1731,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.081893,0.000264,"POLYGON ((130.83681 -12.45376, 130.83684 -12.4...",130.843737,-12.461860
1,0810,0810,AUS,Australia,24.4283,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.241859,0.002031,"POLYGON ((130.89986 -12.36567, 130.89875 -12.3...",130.876652,-12.369772
2,0812,0812,AUS,Australia,35.8899,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.278788,0.002983,"POLYGON ((130.91915 -12.40786, 130.91831 -12.4...",130.914502,-12.377662
3,0820,0820,AUS,Australia,39.0642,http://linked.data.gov.au/dataset/asgsed3/POA/...,0.409134,0.003248,"POLYGON ((130.85260 -12.43994, 130.85089 -12.4...",130.864975,-12.423396
4,0822,0822,AUS,Australia,150775.8030,http://linked.data.gov.au/dataset/asgsed3/POA/...,90.601831,12.564238,"MULTIPOLYGON (((136.56648 -12.08392, 136.56704...",132.883720,-13.020916


In [8]:
poa_geo_info = poa_gdf[['POA_CODE21', 'latitude','longitude']].drop_duplicates('POA_CODE21')

In [9]:
poa_geo_info['POA_CODE21'] = poa_geo_info['POA_CODE21'].astype('int')
poa_geo_info.rename(columns = {'POA_CODE21': 'postcode'}, inplace = True)

In [10]:
poa_geo_info.head()

,postcode,latitude,longitude
0,800,-12.461860,130.843737
1,810,-12.369772,130.876652
2,812,-12.377662,130.914502
3,820,-12.423396,130.864975
4,822,-13.020916,132.883720


In [11]:
poa_geo_info

,postcode,latitude,longitude
0,800,-12.461860,130.843737
1,810,-12.369772,130.876652
2,812,-12.377662,130.914502
3,820,-12.423396,130.864975
4,822,-13.020916,132.883720
...,...,...,...
2636,7466,-42.078782,145.619082
2637,7467,-42.078073,145.545180
2638,7468,-42.166863,145.376529
2639,7469,-41.565570,145.272140


- External Postcode Data

In [12]:
external_postcode = pd.read_csv('../data/tables/external_postcode.csv')

In [13]:
external_postcode.head()

,postcode,place_name,state_name,state_code,latitude,longitude,accuracy
0,200,Australian National University,Australian Capital Territory,ACT,-35.2777,149.1189,1.0
1,221,Barton,Australian Capital Territory,ACT,-35.3049,149.1412,4.0
2,2540,Wreck Bay,Australian Capital Territory,ACT,-35.1627,150.6907,4.0
3,2540,Hmas Creswell,Australian Capital Territory,ACT,-35.0280,150.5501,3.0
4,2540,Jervis Bay,Australian Capital Territory,ACT,-35.1333,150.7000,4.0


In [14]:
postcode_list = list(external_postcode.drop_duplicates(['postcode'])['postcode'])

In [15]:
postcode_lat_mean = external_postcode.groupby(['postcode'])["latitude"].agg(["mean"])
postcode_lat_mean = postcode_lat_mean.reset_index()
postcode_lat_mean.rename(columns = {'mean': 'latitude'}, inplace = True)

In [16]:
postcode_lat_mean

,postcode,latitude
0,200,-35.2777
1,221,-35.3049
2,800,-12.4611
3,801,-12.4611
4,803,-12.4348
...,...,...
3307,9023,-27.4679
3308,9464,-33.8816
3309,9726,-33.8816
3310,9943,-21.1534


In [17]:
postcode_long_mean = external_postcode.groupby(['postcode'])["longitude"].agg(["mean"])
postcode_long_mean = postcode_long_mean.reset_index()
postcode_long_mean.rename(columns = {'mean': 'longitude'}, inplace = True)

In [18]:
postcode_long_mean

,postcode,longitude
0,200,149.1189
1,221,149.1412
2,800,130.8418
3,801,130.8418
4,803,130.7443
...,...,...
3307,9023,153.0281
3308,9464,150.9844
3309,9726,150.9844
3310,9943,149.1655


In [19]:
external_lat_and_long = pd.merge(postcode_lat_mean, postcode_long_mean, on='postcode',how='inner')

In [20]:
external_lat_and_long

,postcode,latitude,longitude
0,200,-35.2777,149.1189
1,221,-35.3049,149.1412
2,800,-12.4611,130.8418
3,801,-12.4611,130.8418
4,803,-12.4348,130.7443
...,...,...,...
3307,9023,-27.4679,153.0281
3308,9464,-33.8816,150.9844
3309,9726,-33.8816,150.9844
3310,9943,-21.1534,149.1655


- Postcode in Consumer Dataset

In [21]:
consumer_df = pd.read_parquet('../data/curated/new_consumer_data.parquet')

In [22]:
consumer_df.loc[:,'postcode'] = consumer_df.loc[:,'postcode'].astype('int')
consumer_df.dtypes

consumer_id    object
name           object
address        object
state          object
postcode        int64
gender         object
user_id         int64
dtype: object

In [23]:
conn_by_postcode = pd.merge(consumer_df,poa_geo_info, on='postcode', how='left')

In [24]:
conn_by_postcode.head()

,consumer_id,name,address,state,postcode,gender,user_id,latitude,longitude
0,1195503,Yolanda Williams,413 Haney Gardens Apt. 742,WA,6935,Female,1,NaN,NaN
1,179208,Mary Smith,3764 Amber Oval,NSW,2782,Female,2,-33.721669,150.379736
2,1194530,Jill Jones MD,40693 Henry Greens,NT,862,Female,3,-18.075204,135.835705
3,154128,Lindsay Jimenez,00653 Davenport Crossroad,NSW,2780,Female,4,-33.702502,150.309864
4,712975,Rebecca Blanchard,9271 Michael Manors Suite 651,WA,6355,Female,5,-33.032428,119.298440


In [25]:
temp = conn_by_postcode[conn_by_postcode.latitude.isna()]
print( f'There are {len(conn_by_postcode)} data, and {len(temp)} of them have no null value.')

There are 499999 data, and 83181 of them have no null value.


In [26]:
null_df = temp.drop_duplicates('postcode').reset_index()
null_df

,index,consumer_id,name,address,state,postcode,gender,user_id,latitude,longitude
0,0,1195503,Yolanda Williams,413 Haney Gardens Apt. 742,WA,6935,Female,1,NaN,NaN
1,17,80965,Michael Burnett,89400 Torres Fort,NSW,1109,Male,18,NaN,NaN
2,22,662940,Cody Scott,513 Davis Parks,WA,6849,Male,23,NaN,NaN
3,29,1360385,Jason Brown,15132 Thomas Crossroad,NSW,1114,Undisclosed,30,NaN,NaN
4,39,1312467,Katelyn Hart,7247 Allison Ville,NSW,1825,Female,40,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
522,15548,762837,Christopher Jones,512 Lee Hills,WA,6946,Male,15549,NaN,NaN
523,15551,1114154,Alexis Roberts,40530 Boyd Wall,NSW,1139,Female,15552,NaN,NaN
524,16334,381811,Jason Ball,541 Sandra Bridge Apt. 955,WA,6892,Male,16335,NaN,NaN
525,16500,1430152,Matthew Stephens,324 Stephens Squares,WA,6847,Male,16501,NaN,NaN


In [36]:
for i in null_df.index:
    for j in external_lat_and_long.index:
        if null_df['postcode'][i] == external_lat_and_long['postcode'][j]:
            null_df['latitude'][i] = external_lat_and_long['latitude'][j]
            null_df['longitude'][i] = external_lat_and_long['longitude'][j]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [37]:
filled_df = null_df[['postcode', 'latitude', 'longitude']]
filled_df.head()

,postcode,latitude,longitude
0,6935,-31.9000,115.9667
1,1109,-33.8678,151.2073
2,6849,-31.9333,115.8333
3,1114,-33.8678,151.2073
4,1825,-33.8644,151.0397


In [38]:
semi_pos_geo_data = pd.concat([poa_geo_info, filled_df])

In [39]:
semi_conn_by_postcode = pd.merge(semi_pos_geo_data,consumer_df, on='postcode', how='left')

In [40]:
final_pos_geo_data = semi_conn_by_postcode[['postcode', 'latitude', 'longitude', 'state']].drop_duplicates(['postcode']).reset_index(drop=True)

In [41]:
final_pos_geo_data

,postcode,latitude,longitude,state
0,800,-12.461860,130.843737,NT
1,810,-12.369772,130.876652,NT
2,812,-12.377662,130.914502,NT
3,820,-12.423396,130.864975,NT
4,822,-13.020916,132.883720,NT
...,...,...,...,...
3163,6946,-31.750000,115.800000,WA
3164,1139,-33.867800,151.207300,NSW
3165,6892,-31.958700,115.871100,WA
3166,6847,-31.952200,115.861400,WA


- Connect pos_data with poa_data

In [42]:
sa2_sf = gpd.read_file('../data/tables/external_SA2/SA2_2021_AUST_GDA2020.shp')
sa2_sf['geometry'] = sa2_sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sa2_gdf = gpd.GeoDataFrame(sa2_sf)
sa2_gdf = sa2_gdf.dropna()
bound = sa2_gdf['geometry'].bounds
sa2_geo = sa2_gdf.join(bound)

In [43]:
sa2_geo_info = sa2_geo[['SA2_CODE21', 'geometry']].drop_duplicates('SA2_CODE21')

In [44]:
def match_sa2_and_postcode(sa2_df, pos_df):
    # pos_new_df = pos_df.reindex(columns=[*pos_df.columns.tolist(), 'SA2_CODE21'], fill_value=0)
    for i in pos_df.index:
        for j in sa2_df.index:
            if ((pos_df['latitude'][i] >= sa2_df['miny'][j])& (pos_df['latitude'][i] <= sa2_df['maxy'][j])
                &(pos_df['longitude'][i] >= sa2_df['minx'][j]) & (pos_df['longitude'][i] <= sa2_df['maxx'][j])):
                pos_df.loc[i, 'SA2_CODE21'] = sa2_df['SA2_CODE21'][j]
    return pos_df

In [45]:
mix_df = match_sa2_and_postcode(sa2_geo, final_pos_geo_data)

In [46]:
mix_df = mix_df.dropna()

In [47]:
final_df = pd.merge(mix_df, sa2_geo_info, on='SA2_CODE21',how='inner')

In [48]:
final_df.head(5)

,postcode,latitude,longitude,state,SA2_CODE21,geometry
0,800,-12.461860,130.843737,NT,702031057,"MULTIPOLYGON (((130.01952 -13.17563, 130.01928..."
1,810,-12.369772,130.876652,NT,702031057,"MULTIPOLYGON (((130.01952 -13.17563, 130.01928..."
2,812,-12.377662,130.914502,NT,702031057,"MULTIPOLYGON (((130.01952 -13.17563, 130.01928..."
3,820,-12.423396,130.864975,NT,702031057,"MULTIPOLYGON (((130.01952 -13.17563, 130.01928..."
4,828,-12.428255,130.939585,NT,702031057,"MULTIPOLYGON (((130.01952 -13.17563, 130.01928..."


In [49]:
final_df.dtypes

postcode         int64
latitude       float64
longitude      float64
state           object
SA2_CODE21      object
geometry      geometry
dtype: object

In [50]:
final_df = final_df.dropna()

In [53]:
final_geo_df= gpd.GeoDataFrame(final_df)

In [54]:
final_geo_df.to_file('../data/curated/geo_sa2_pos.shp',
           driver='ESRI Shapefile',
           encoding='utf-8')

In [55]:
final_geo_df.to_parquet('../data/curated/sa2_pos_geometry.parquet')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
  """Entry point for launching an IPython kernel.
